In [39]:

#----- imports --------
import tqdm
import torch
# import wandb
import os
import tokenizers


device= 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)
assert device == 'cuda', "This notebook is not optimized for CPU"

config = {
    "learning_rate": 1e-3,
    "eval_interval": 300,
    "max_iters": 60000, 
    "H": 32, # hidden dimension size
    "B": 64,
    "T": 256,
    "C": 256,
    "feedforward_factor": 3,
    "n_heads": 8,
    "dropout": 0.0,
    "l2_penalty": 0.0,
    "n_layers": 12,
    "tokenizer_vocab_size": 2**13,
    # "git_hash": os.popen("git rev-parse HEAD").read().strip()
}

# initial
for k,v in config.items():
    locals ()[k] = v


#wandb.init(
#    project = "tinystories",
#    config = config,
#)

In [6]:
os.chdir('multilingual_tinystories')

In [7]:
def load_sharded_story(shard_no):
    return torch.load(f'tokenized/tokenized-{shard_no}.pt')

In [10]:
# load the tokenized stories in parallel using threads
# this is faster than loading them sequentially
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor() as pool:
    stories = list(tqdm.tqdm(pool.map(load_sharded_story, range(11)), total=11))



  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [05:11<00:00, 28.34s/it]


In [13]:
all_stories = []
for story in stories:
    all_stories.extend(story)
    

In [15]:
print("length of dataset in stories: ", len(all_stories))
print("length of stories in tokens", sum(len(story) for story in all_stories))

length of dataset in stories:  5304143
length of stories in tokens 919732177


In [40]:
# plot a histogram of the lengths of the stories
import matplotlib.pyplot as plt
# plt.hist([len(story) for story in all_stories[:10000]], bins=50)
num_stories_to_check = 1_000_000
num_long = sum(len(story) > T for story in all_stories[:num_stories_to_check])
print(
    f"# stories longer than {T} : {num_long} out of {num_stories_to_check}, {num_long/num_stories_to_check:.2%}")

# stories longer than 256 : 47531 out of 1000000, 4.75%


In [81]:
tokenizer = tokenizers.ByteLevelBPETokenizer(
    "./tiny-stories-spanish-bpe-vocab.json", 
    "./tiny-stories-spanish-bpe-merges.txt"
)
chars_per_token = 3.9 # hack

In [44]:

def encode(text):
    return tokenizer.encode(text).ids
def decode(encoded_text):
    return tokenizer.decode(encoded_text)

from tqdm import tqdm

def batch_encode(text, batch_size):
    tokens = []
    for i in tqdm(range(0, len(text), batch_size)):
        tokens.extend(encode(text[i:i+batch_size]))
    return tokens


hello_encoded = encode("Hola")
print(hello_encoded)
print(decode(hello_encoded))
vocab_size = tokenizer.get_vocab_size()
print("vocab size: ", vocab_size)
print('first story decoded: ', decode(all_stories[0].tolist()))
PADDING_TOKEN_IDX= encode(" ")[0]

[573]
Hola
vocab size:  8192
first story decoded:  Un día, un niño llamado Leo jugó con cartas. Él tomó las cartas y las puso en un montón. Después, él las puso en otro montón. ¡Él inventó un nuevo juego! 

Leo miró las cartas. Él vio un gato. "¡Miau!" dijo el gato. Después, él vio un perro. "¡Guau!" dijo el perro. Leo sonrió. Él pensó que su juego era divertido. 

Después de jugar, Leo se fue a la cama. Él soñaba con el gato y el perro. En su sueño, el gato y el perro corrieron juntos. Ellos eran felices.

¡De repente! Leo despertó. Él sintió que algo no estaba bien. Él fue a la ventana. Él vio al gato y al perro jugando afuera. ¡El sueño se había hecho realidad! 

Leo corrió afuera y jugó con el gato y el perro. Él jugó despacio. Él no quería asustarlos. Él era feliz. Él sabía que su sueño había sido real. 



In [24]:
n = int(0.9*len(all_stories))

train_data = all_stories[:n]
val_data = all_stories[n:]

In [26]:
len(train_data)

4773728

In [15]:
x = train_data[:T]
y = train_data[1:T+1]
for t in range(T):
    context = x[:t+1]
    target = y[t]
    # print("when we see the text", context, "we predict the next character is", target)

In [52]:
torch.manual_seed(1339)

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data), (B,)) 
    # ix = [i for i in range(B)]
    
    x = torch.full((B, T), PADDING_TOKEN_IDX, dtype=torch.long)
    y = torch.full((B, T), PADDING_TOKEN_IDX, dtype=torch.long)

    for sequence_index, random_story_index in enumerate(ix):
        story = data[random_story_index].long()[:T - 1]
        x[sequence_index][1: story.shape[0]+1] = story
        y[sequence_index][: story.shape[0]] = story

        
    return x, y

xb, yb = get_batch('train')

print(xb[0])
print(yb[0])


tensor([ 220,  463,  281,  420, 1170,   13,  284,  410,  337,  503,  403,  945,
        1213,   13,  452, 1082,  330,  436, 3540,   13,  371,  350,  330,  403,
         381,  302,   25,  285,  639,   11,  292,  299, 1082,  330,  436, 3540,
         315,  403,  352,  394,  749,  553, 3540,   13, 1094,  348, 5732,   13,
         301,  198,  639,  311,  889,  323,  325, 1479,  306,  837,  436, 3540,
          13, 1094,  348, 4641,   13,  375,  394,  355, 1213,   13,  403, 1369,
         291, 1479,  331,  553, 2323,   13,  285,  400,  807,  749, 1243, 3540,
         822,  302,  403,   13,  301,  198,  444,  350,  330,  403,  381,  302,
          25,  285,  639,   11,  292,  347, 3830,  749, 1543, 3540,  315,  403,
         318,  416,   13,  220,  512,  796,  651,  351,  388,  351,  749,  553,
        3540,   13,  375, 1369,  436, 3540,  331,  325, 1334,   13,  301,  198,
         639, 1802,  261,  749,  553, 3540,   13,  452, 2489,   11,  588,  796,
         484,  792,   13, 1847, 2541,   

In [65]:

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class Head(nn.Module):
    '''One Head of self-attention'''
    def __init__(self, H):
        super().__init__()
        self.query = nn.Linear(C, H, bias=False)
        self.key = nn.Linear(C, H, bias=False)
        self.value = nn.Linear(C, H, bias=False)
        # self.output = nn.Linear(H, C, bias=False) # output matrix
        self.register_buffer('tril', torch.tril(torch.ones(T, T)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        query_vectors = self.query(x)
        key_vectors = self.key(x)


        # Attention masking(so we can't look into the past):

        tril = self.tril
        wei = torch.zeros(T, T) 
        wei = wei.masked_fill(tril == 0, float('-inf')) # set the upper triangular to -inf
        # xbow = wei @ x # apply the mask to the input, bag of words because simple avg.

        # multiply the two to get the attention weights
        attention_pattern = query_vectors @ key_vectors.transpose(-2, -1) # T, T
        attention_pattern = attention_pattern / (H ** 0.5) # scale the attention pattern for numerical stability
        attention_weights = F.softmax(attention_pattern + wei, dim=-1) # T, T (the row dimension is the query)
        attention_weights = self.dropout(attention_weights)

        value_vectors = self.value(x) # the direction we should go in the embedding space for each token (ie more blue) T, H

        # apply the attention weights to the value vectors
        context = attention_weights @ value_vectors # T, H

        # project back into original space from value space
        # return self.output(context)
        return context

x = torch.randn(B,T,C)
head = Head(H)
# head(x)

In [66]:
class MultiHeadAttention(nn.Module):
    '''Multiple heads of self-attention'''
    def __init__(self, H, C, n_heads): # H is head embedding space size, n_heads is number of heads
        super().__init__()
        self.heads = nn.ModuleList([Head(H) for _ in range(n_heads)])
        self.combine_heads = nn.Linear(H*n_heads, C)
        self.dropout = nn.Dropout(dropout)


    def forward(self,x):
        x = torch.cat([head(x) for head in self.heads], dim=-1)
        x = self.combine_heads(x)  # T, C
        return self.dropout(x)

In [67]:
head = MultiHeadAttention(H, C, n_heads)
head.heads[0].forward(x).shape


torch.Size([64, 256, 32])

In [68]:
class FeedForward(nn.Module):
    '''Feed-forward neural network'''
    def __init__(self, C):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(C, C * feedforward_factor),
            nn.ReLU(),
            nn.Linear(C * feedforward_factor, C),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [69]:
class LayerNorm(nn.Module):
    '''Layer normalization'''
    def __init__(self, C, use_affine=True):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(C)) if use_affine else None
        self.beta = nn.Parameter(torch.zeros(C)) if use_affine else None

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        if self.gamma is not None and self.beta is not None:
            return self.gamma * (x - mean) / (std + 1e-6) + self.beta
        else:
            return (x - mean) / (std + 1e-6)

In [70]:
class Block(nn.Module):
    '''Transformer block'''
    def __init__(self, H, C, n_heads):
        super().__init__()
        self.attention = MultiHeadAttention(H, C, n_heads)
        self.ff = FeedForward(C)
        self.norm1 = LayerNorm(C, use_affine=True)
        self.norm2 = LayerNorm(C, use_affine=True)

    def forward(self, x):
        x = x + self.attention(self.norm1(x))
        x = x + self.ff(self.norm2(x))
        return x

In [71]:
class GPT(nn.Module):

    def __init__(self, n_layers):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, C) 
        self.position_embedding_table = nn.Embedding(T, C)
        self.lm_head = nn.Linear(C, vocab_size)
        self.layers = nn.ModuleList([Block(H, C, n_heads) for _ in range(n_layers)])
        self.block = nn.ModuleList([Block(H, C, n_heads)])
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        token_emb = self.token_embedding_table(idx) # batch_dim, sequence_dim, embedding_dim
        pos_emb = self.position_embedding_table(torch.arange(T))
        x = token_emb + pos_emb # token identities and positions contained

        for layer in self.layers:
            x = layer(x)

        logits = self.lm_head(x) # batch_dim, sequence_dim, vocab_size

        batch_dim, sequence_dim, embedding_dim = logits.size()

        # loss = F.cross_entropy(logits, targets) this won't work because we need 1d logits and 1d targets
        # one-hot-vectors are a line in the x-dimension, so the shape of shape of the logits should be (-1, vocab_size).

        if targets is None:
            return logits, None
        else:
            # a list of all the predictions, reguardles of batch.
            # xdim: probabilities of each character in the vocab (embedding_dim=vocab_size)
            # ydim: all predictions for all batches flattened (batch_dim*sequence_dim)
            logits_loss_view = logits.view(-1, vocab_size) 
            # targets loss view
            # xdim: all targets for all batches flattened (batch_dim*sequence_dim)
            # so this would be like, [1,4,5,1,2,3, ...]
            # where each number is the correct next index of the one hot vector
            targets_loss_view = targets.view(-1)
            loss = F.cross_entropy(logits_loss_view, targets_loss_view)
            return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx[:,-T:])
            # get the predictions of the last token
            last_token_logits = logits[:, -1, :] # all batches, last token, all probabilities
            # softmax to get probabilities
            probabilities = F.softmax(last_token_logits, dim=-1)
            # sample from the probabilities
            next_token = torch.multinomial(probabilities, num_samples=1)
            # add the new token to the idx tensor
            idx = torch.cat((idx, next_token), dim=1)
        return idx
    def prompt_model(self, prompt, max_new_tokens, temperature=0.5):
        autoregressive_seq = encode(prompt)
        for _ in range(max_new_tokens):
            prediction_index = len(autoregressive_seq)-1

            model_input = torch.tensor(autoregressive_seq)
            
            while model_input.shape[0] < T:
                pad_token = torch.tensor(encode("\n"))
                model_input = torch.cat((model_input, pad_token), dim=0)

            model_input
            model_input = model_input.unsqueeze(0)

            logits, loss = model(model_input)
            prediction_token = logits[:, prediction_index, :] / temperature
            probabilities = F.softmax(prediction_token, dim=-1)
            next_token = torch.multinomial(probabilities, num_samples=1)
            next_token = next_token.item()

            autoregressive_seq.append(next_token)
        # get the autoregressive sequence
        return decode(autoregressive_seq)



    

model = GPT(n_layers)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)




test_idx = torch.zeros(1, T).long()
model.forward(idx=test_idx)
# decode(model.generate(idx=test_idx, max_new_tokens=100)[0].tolist())

torch.Size([64, 256, 8192])
tensor(9.4822, device='cuda:0', grad_fn=<NllLossBackward0>)


(tensor([[[ 1.2796,  0.2360,  0.4649,  ..., -0.8759,  0.0897,  0.0632],
          [ 2.1053, -0.8288,  0.3834,  ..., -1.1195,  0.1282, -0.5274],
          [ 2.2574,  0.5220,  0.4413,  ..., -1.4343, -1.8042, -1.5558],
          ...,
          [ 2.5515, -0.5823,  0.1534,  ..., -1.5195, -1.1673, -0.1516],
          [ 1.2516, -0.4528,  0.2085,  ..., -0.0323, -0.3258, -0.9920],
          [ 2.0084, -1.3079,  0.2250,  ...,  0.0589, -1.0247, -1.3288]]],
        device='cuda:0', grad_fn=<ViewBackward0>),
 None)

In [72]:
model

GPT(
  (token_embedding_table): Embedding(8192, 256)
  (position_embedding_table): Embedding(256, 256)
  (lm_head): Linear(in_features=256, out_features=8192, bias=True)
  (layers): ModuleList(
    (0-11): 12 x Block(
      (attention): MultiHeadAttention(
        (heads): ModuleList(
          (0-7): 8 x Head(
            (query): Linear(in_features=256, out_features=32, bias=False)
            (key): Linear(in_features=256, out_features=32, bias=False)
            (value): Linear(in_features=256, out_features=32, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (combine_heads): Linear(in_features=256, out_features=256, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=256, out_features=768, bias=True)
          (1): ReLU()
          (2): Linear(in_features=768, out_features=256, bias=True)
          (3): Dropout(p=0.0, inplace=False)
  

In [73]:
# get the number of parameters in the model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("number of parameters in the model: ", count_parameters(model))

number of parameters in the model:  12817664


In [75]:
# logits, loss = self(idx[:,-T:])

idx = torch.zeros(1, 1).long()
idx[:,-T:]

tensor([[0]], device='cuda:0')

In [76]:
model.token_embedding_table.weight.device

device(type='cuda', index=0)

In [77]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



In [78]:
eval_iters = 10
eval_interval = 300
@torch.no_grad()
def estimate_loss(is_last=False):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        real_iters = eval_iters
        if is_last and split == 'val':  # increase last eval to mitigate noise
            real_iters *= 10 
        losses = torch.zeros(real_iters)
        for k in range(real_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean() / chars_per_token
    model.train()
    return out
    

In [79]:
# get the number of parameters
n_params = sum(p.numel() for p in model.parameters())
parameter_to_data_ratio = n_params / len(train_data)
print(f"{parameter_to_data_ratio=}")

parameters = []
for name, param in model.named_parameters():
    parameters.append({"name": name, "params": param.numel()})

# sort parameters by size
sorted_parameters = sorted(parameters, key=lambda x: x["params"], reverse=True)
for p in sorted_parameters:
    print(f"{p['name']}: {p['params']}")

parameter_to_data_ratio=2.6850428009304257
token_embedding_table.weight: 2097152
lm_head.weight: 2097152
layers.0.ff.net.0.weight: 196608
layers.0.ff.net.2.weight: 196608
layers.1.ff.net.0.weight: 196608
layers.1.ff.net.2.weight: 196608
layers.2.ff.net.0.weight: 196608
layers.2.ff.net.2.weight: 196608
layers.3.ff.net.0.weight: 196608
layers.3.ff.net.2.weight: 196608
layers.4.ff.net.0.weight: 196608
layers.4.ff.net.2.weight: 196608
layers.5.ff.net.0.weight: 196608
layers.5.ff.net.2.weight: 196608
layers.6.ff.net.0.weight: 196608
layers.6.ff.net.2.weight: 196608
layers.7.ff.net.0.weight: 196608
layers.7.ff.net.2.weight: 196608
layers.8.ff.net.0.weight: 196608
layers.8.ff.net.2.weight: 196608
layers.9.ff.net.0.weight: 196608
layers.9.ff.net.2.weight: 196608
layers.10.ff.net.0.weight: 196608
layers.10.ff.net.2.weight: 196608
layers.11.ff.net.0.weight: 196608
layers.11.ff.net.2.weight: 196608
block.0.ff.net.0.weight: 196608
block.0.ff.net.2.weight: 196608
position_embedding_table.weight: 65

In [87]:
import tqdm
num_params = sum([p.numel() for p in model.parameters()])

dump_model_interval = 1000

for steps in tqdm.tqdm(range(max_iters)):
    xb, yb = get_batch('train')
    # loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    # l2 regularization
    # l2 = sum(p.pow(2).sum() for p in model.parameters()) / num_params
    loss = loss # + l2 * l2_penalty

    loss.backward()
    optimizer.step()
    if steps % eval_interval == 0:
        losses = estimate_loss()
        # wandb.log({"train": losses['train'].item(), "val": losses['val'].item(), "l2":l2})
        print({"tIain": losses['train'].item(), "val": losses['val'].item()})
    if steps % dump_model_interval == 0 and steps > 0:
        model_no = steps // dump_model_interval
        torch.save(model.state_dict(), f'tiny-stories-model-{model_no}.pt')

losses = estimate_loss(is_last=True)
#wandb.log({"train": losses['train'].item(), "val": losses['val'].item()})
#wandb.finish()


  0%|          | 1/60000 [00:02<40:54:15,  2.45s/it]

{'tIain': 0.23402966558933258, 'val': 0.2347315102815628}


  1%|          | 301/60000 [01:58<17:00:04,  1.03s/it]

{'tIain': 0.23102733492851257, 'val': 0.23457872867584229}


  1%|          | 601/60000 [03:55<17:05:31,  1.04s/it]

...

{'tIain': 0.21533919870853424, 'val': 0.21453677117824554}


 64%|██████▍   | 38401/60000 [4:09:53<6:12:55,  1.04s/it]

{'tIain': 0.21800558269023895, 'val': 0.21224913001060486}


 64%|██████▍   | 38585/60000 [4:11:04<2:19:21,  2.56it/s]


KeyboardInterrupt: 

In [74]:
estimate_loss()

{'train': tensor(0.4513, device='cuda:0'),
 'val': tensor(0.4492, device='cuda:0')}

In [38]:
# save model
torch.save(model.state_dict(), 'tiny-stories-model.pt')

In [52]:
# load the model
model.load_state_dict(torch.load('tiny-stories-model.pt'))


<All keys matched successfully>

In [79]:
encode("\n")

[198]

In [92]:
print(model.prompt_model(" Un día, un niño", 200, temperature=0.5))

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [34]:
test_idx = torch.zeros(1, T).long() * 198
print(decode(
    model.generate(idx=test_idx, max_new_tokens=C)[0].tolist()
)[T:])




The first room was filled with joy. He raised wide and wide-bye and skipped on to the chest with a big smile. Every nowe on the same head. Aneagull was the most beautiful and quiet place! Aeon lilyolog opened the window andœOf course clapped in the lush road. Forward was nothing! Today had been great! She was so happy and so excited that she cried out that they couldn't stop until they gave up. She had done it!"
"Once upon a time there was an angry volcano. Daisy shouted loud and refused to do anything.

Then, bowl was filled with her anger from the volcano. She wished that the volcano would be more careful.

But, the volcano didn't ignorant. Gootators were too bossy. It wanted to cause trouble and become angry."
"Once upon a time, there was a man who wanted to go for a ride. He was so excited! â€œThomas!â€ he shouted. â€œLet's go!â€

Thomas and the man got on the bus. The billboard opened. Out of the window light came on Max but he got so excited. The picture in the picture was the